In [1]:
from bs4 import BeautifulSoup as Soup
from pandas import DataFrame
import requests

In [2]:
response = requests.get('https://www.hockey-reference.com/teams/NJD/index.html')

In [3]:
print(response.text)


<!DOCTYPE html>
<html data-version="klecko-" data-root="/home/hr/build" lang="en" class="no-js" >
<head>
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />
    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202307191" />
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script type="text/javascript" async=true>
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAM

In [4]:
soup = Soup(response.text)

In [5]:
tables = soup.find_all('table')

In [6]:
len(tables)

2

In [8]:
tables[1]

<table class="sortable stats_table" data-cols-to-freeze=",2" id="roster">
<caption>Roster Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr>
<th aria-label="No." class="poptip sort_default_asc center" data-stat="number" data-tip="Sweater Number" scope="col">No.</th>
<th aria-label="Player" class="poptip sort_default_asc left" data-stat="player" scope="col">Player</th>
<th aria-label="flag" class="poptip center" data-stat="flag" scope="col">Flag</th>
<th aria-label="Pos" class="poptip sort_default_asc center" data-stat="pos" scope="col">Pos</th>
<th aria-label="Age" class="poptip sort_default_asc center" data-stat="age" data-tip="As of Jan. 31 of the season in question. Or as of the date of the game listed. Age is Years-Days." scope="col">Age</th>
<th aria-label="Ht" class="poptip sort_default_asc center" data-stat="height" scope="col">Ht</th>
<th aria-label="Wt" class="poptip sort_default_asc center" data-stat="wei

In [9]:
roster = tables[1]

In [10]:
rows = roster.find_all('tr')

In [11]:
first_data_row = rows[1]

In [12]:
first_data_row.find_all('td')

[<td class="left" csk="Bahl,Kevin" data-append-csv="bahlke01" data-stat="player"><a href="/players/b/bahlke01.html">Kevin Bahl</a></td>,
 <td class="right" data-stat="flag"><span class="f-i f-ca" style="">ca</span></td>,
 <td class="center" csk="4:Bahl:Kevin" data-stat="pos">D</td>,
 <td class="center" data-stat="age">23</td>,
 <td class="right" csk="78.0" data-stat="height">6-6</td>,
 <td class="right" data-stat="weight">230</td>,
 <td class="center" data-stat="shoots_and_catches">L/-</td>,
 <td class="right" data-stat="years_experience">3</td>,
 <td class="left" csk="20000627" data-stat="birth_date">June 27, 2000</td>,
 <td class="right" data-stat="salary"> $900,000</td>,
 <td class="left" data-stat="draft">2018 ARI 2nd (55)</td>]

In [13]:
[str(x.string) for x in first_data_row.find_all('td')]

['Kevin Bahl',
 'ca',
 'D',
 '23',
 '6-6',
 '230',
 'L/-',
 '3',
 'June 27, 2000',
 ' $900,000',
 '2018 ARI 2nd (55)']

In [14]:
def parse_row(row):
    """
    take in a tr tag and get the data out of it in the form of strings"""
    
    return [str(x.string) for x in row.find_all('td')]

In [15]:
list_of_parsed_rows = [parse_row(row) for row in rows[1:]]

In [16]:
df = DataFrame(list_of_parsed_rows)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,Kevin Bahl,ca,D,23,6-6,230,L/-,3,"June 27, 2000","$900,000",2018 ARI 2nd (55)
1,Nathan Bastian,ca,F,26,6-4,205,R/-,4,"December 6, 1997","$1,350,000",2016 NJD 2nd (41)
2,Shane Bowers,ca,F,24,6-2,186,L/-,1,"July 30, 1999","$125,000",2017 OTT 1st (28)
3,Jesper Bratt,se,F,25,5-10,175,L/-,6,"July 30, 1998","$5,000,000",2016 NJD 6th (162)
4,Graeme Clarke,us,F,22,6-0,175,R/-,R,"April 24, 2001","$70,000",2019 NJD 3rd (80)


In [18]:
df.columns = ['Player', 'Nationality', 'Position', 'Age', 'Height', 'Weight', 'Hand', 'Experience', 'Birthday', 'Cap Hit','Draft']

In [19]:
df.sample(3)

,Player,Nationality,Position,Age,Height,Weight,Hand,Experience,Birthday,Cap Hit,Draft
5,Kyle Criscuolo,us,F,31,5-9,175,R/-,3,"May 5, 1992","$330,000",None
12,None,ch,F,25,6-1,175,L/-,6,"January 4, 1999","$7,750,000",2017 NJD 1st (1)
10,Dougie Hamilton,ca,D,30,6-6,230,R/-,11,"June 17, 1993","$10,600,000",2011 BOS 1st (9)


In [20]:
### Get Rid of Symbols
def parse_row_clean(row):
    return [str(x.string)
           .replace('$','')
           .replace(',','') for x in row.find_all('td')]

In [21]:
roster_clean = DataFrame([parse_row_clean(row) for row in rows[1:]])

In [22]:
df.head()

,Player,Nationality,Position,Age,Height,Weight,Hand,Experience,Birthday,Cap Hit,Draft
0,Kevin Bahl,ca,D,23,6-6,230,L/-,3,"June 27, 2000","$900,000",2018 ARI 2nd (55)
1,Nathan Bastian,ca,F,26,6-4,205,R/-,4,"December 6, 1997","$1,350,000",2016 NJD 2nd (41)
2,Shane Bowers,ca,F,24,6-2,186,L/-,1,"July 30, 1999","$125,000",2017 OTT 1st (28)
3,Jesper Bratt,se,F,25,5-10,175,L/-,6,"July 30, 1998","$5,000,000",2016 NJD 6th (162)
4,Graeme Clarke,us,F,22,6-0,175,R/-,R,"April 24, 2001","$70,000",2019 NJD 3rd (80)


In [25]:
int_cols = [x for x in roster_clean.columns if x == 'Cap Hit' or  x == 'Age' ]

In [26]:
roster_clean[int_cols] = roster_clean[int_cols].astype(int)

In [27]:
roster_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       34 non-null     object
 1   1       34 non-null     object
 2   2       34 non-null     object
 3   3       34 non-null     object
 4   4       34 non-null     object
 5   5       34 non-null     object
 6   6       34 non-null     object
 7   7       34 non-null     object
 8   8       34 non-null     object
 9   9       34 non-null     object
 10  10      34 non-null     object
dtypes: object(11)
memory usage: 3.0+ KB
